In [1]:
import pandas as pd
import numpy as np
import os
import glob

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from helpers import *

# Data prep

Läs in fil

In [3]:
df = pd.read_csv('../indata/ODB_TSV/ODB-utbildningsnivå, ålder, tid och kön 2015-2017.txt', sep='\t', header=1)

Specificera indexering

In [4]:
df = df.set_index(['område', 'utbildningsnivå'])

Vänd ner samtliga kolumner i en kolumn

In [5]:
df = pd.DataFrame(df.stack())

Lyft ut index till kolumner igen

In [6]:
df = df.reset_index()

Sätt beskrivande kolumnnamn

In [7]:
df = df.rename(columns={'level_2': 'ageyeargender', 0: 'value'})

Splitta områdesnamn till endast baskod

In [8]:
df['område'] = df['område'].str.split(' ', 1, expand = True)
df = df.rename(columns={'område': 'BASKOD2010'})
df['BASKOD2010'] = df['BASKOD2010'].astype(str).astype(int)

Byt om `BASKOD2010` till entity-namnet

In [9]:
df = baskod2010tobasomrade(df)

Splitta ut `ageyeargender` till fyra separata kolumner

In [10]:
df['agegroup'], df['kasta'], df['year'], df['gender'] = df['ageyeargender'].str.split(' ').str

Kasta kolumner som inte längre är relevanta

In [11]:
df = df.drop('kasta', 1)
df = df.drop('ageyeargender', 1)

Specificera ordningen på kolumnerna

In [12]:
df = df[['basomrade','year','utbildningsnivå', 'gender', 'agegroup', 'value']]

Specificera indexering

In [13]:
df = df.set_index(['basomrade'])

Slå samman åldersgrupper till en `25-64` grupp.

In [14]:
df = df[df['agegroup'] != '20-24']

In [15]:
df = df.groupby(['basomrade', 'year', 'utbildningsnivå', 'gender']).sum().reset_index()

Byt namn på `gender` kolumnen för att följa konventionen i övriga notebooks.

In [16]:
df = df.rename(columns={'gender':'Kön'})

Sätt `Kön` till en pandas category

In [17]:
df['Kön'] = df['Kön'].astype('category')
df['Kön'].cat.categories = ['male', 'female']

# Befolkning 25-64

Beräkna summan av antalet för varje basområde

In [18]:
pop = df.groupby(['basomrade', 'year', 'Kön']).sum().reset_index()

Exportera

In [19]:
concept = 'population_aged_25_64'

In [20]:
appendNewDatapoints(concept, pop)
byGender(concept, pop)

Saved population_aged_25_64 to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--population_aged_25_64--by--basomrade--year.csv

Saved population_aged_25_64_male to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--population_aged_25_64_male--by--basomrade--year.csv

Saved population_aged_25_64_female to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--population_aged_25_64_female--by--basomrade--year.csv

